# ABOUT
- background:
    - in part 1, i acquired all SCSE member names
- for each scse member acquire their author profile Using Google Scholar API
    - steps:
        1. for each author name, perform an author search to find the author_d
        2. using the author_id call the Google Scholar API

In [3]:
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import pickle
import os

In [4]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Tan_Ching_Fhen\Tan_Ching_Fhen.json"
with open(path, "r") as f:
    all_ntu_researchers = json.load(f)

In [5]:
def from_scse(researcher_document):
    keyword = "Computer Science"
    return sum(map(lambda school: keyword in school, researcher_document["schools"]))>0
def get_researcher_names(data):
    return [researcher_document["full_name"] for researcher_document in all_ntu_researchers if from_scse(researcher_document)]
scse_researcher_names = get_researcher_names(all_ntu_researchers)

In [6]:
"""
Flexible name matching such that "Lu Shijian" can match "Shijian Lu" and many other variations
"""
def score(s1, s2):
    scores = [
        fuzz.token_sort_ratio(s1, s2), 
        fuzz.token_set_ratio(s1, s2), 
        fuzz.partial_ratio(s1, s2)
    ]
    return max(scores)

In [7]:
"""
Gets the end point to a person's google scholar profile 
"""
def get_GS_profile_endpoint(name):
    # search SCSE member's name on google scholar
    url = get_scholar_author_search_url(name)
    response = requests.get(url)
    soup  = BeautifulSoup(response.text, "html.parser")
    # check response 200
    assert response.status_code==200
    researcher_body = soup.find("div",{"class":"gsc_1usr"})
    if researcher_body is None:
        print(f"Not Found: {name}")
        return 
    name_found = researcher_body.find("span",{"class":"gs_hlt"}).text
    # check that name matches
    if score(name_found,name)>=89:
        print(f"Matched: {name} - {name_found}")
        endpoint = researcher_body.a['href']
    else:
        print(f"Not Matched: {name} - {name_found}")
        endpoint = None
    return endpoint
"""
URL that searches a google scholar profile 
"""
def get_scholar_author_search_url(name):
    additional_keywords = "Nanyang Technological University"
    mauthors = "+".join(additional_keywords.split()+name.split())
    url = f"https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors={mauthors}&btnG="
    return url   


# Shortening name may help
def alternative_name(name):
    tokens = name.split(" ")
    if len(tokens)==2:
        # remove first name
        tokens.pop(0)
    elif len(tokens)==3:
        # remove middle name
        tokens.pop(1)
    return " ".join(tokens)

### find author_id
- if not found try an alternative name

In [8]:
scse_endpoints = [get_GS_profile_endpoint(name) for name in scse_researcher_names]
for i, (endpoint,name) in enumerate(zip(scse_endpoints,scse_researcher_names)):
    if endpoint is None:
        scse_endpoints[i] = get_GS_profile_endpoint(alternative_name(name))

Matched: Lin Guosheng - Guosheng Lin
Matched: Yu Han - Han Yu
Matched: Liu Weichen - Weichen Liu
Matched: Lam Siew Kei - Siew
Matched: Loke Yuan Ren - Yuan Ren Loke
Matched: Lam Kwok Yan - Kwok
Matched: Guan Cuntai - Cuntai Guan
Matched: Anwitaman Datta - Anwitaman Datta
Matched: Lin Weisi - Weisi Lin
Matched: Liu Ziwei - Ziwei Liu
Matched: Chee Wei Tan - Chee Wei Tan
Matched: Fan Xiuyi - Xiuyi Fan
Matched: Anupam Chattopadhyay - Anupam Chattopadhyay
Matched: Zhao Jun - Jun Zhao
Not Found: Quek Hiok Chai
Matched: Sun Aixin - Aixin Sun 
Matched: A S Madhukumar - Madhukumar
Matched: Deepu Rajan - Deepu Rajan
Matched: Zheng Jianmin - Jianmin Zheng
Matched: Lin Shang-Wei - Shang
Matched: Wentong Cai - Wentong Cai
Matched: Li Yi - Yi Li
Matched: Chen Change Loy - Chen Change Loy
Matched: Cham Tat Jen - Tat
Matched: Kwoh Chee Keong - Chee Keong KWOH
Matched: Smitha K G - Smitha KG
Matched: Tan Rui - Rui Tan
Matched: Cong Gao - Gao Cong
Not Found: Seah Hock Soon
Matched: Luo Jun - Jun Luo 
Ma

In [98]:
from serpapi import GoogleSearch

"""
from google_scholar_author_api, get author profile using author_id
"""
def call_google_scholar_author_api(author_id, api_key = "623ca6f08638abdaecd72efd193971122545dd1d367645fbbee823f29b57b072"):
    params = {
        "engine": "google_scholar_author",
        "hl": "en",
        "author_id": author_id,
        "start": "0",
        "num":1000,
        "sort":"Year",
        "api_key": api_key
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

### call Google Scholar Author API
- use the author_id found

In [ ]:
data_dir = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 2\data"

for endpoint in scse_endpoints:
    if endpoint is None:
        continue
    # get the author_id from the endpoint
    author_id = endpoint.split("=")[-1]
    # call GS API
    results = call_google_scholar_author_api(author_id)
    # save results
    author_name = results['author']["name"].replace('\"',"")
    filename = f"{author_name}.json"
    with open(os.path.join(data_dir,filename), "w") as f:
        json.dump(results, f)
    print(f"{author_name} acquired")
    # with open(path, "r") as f:
    #     tmp = json.load(f)

In [105]:
with open(r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Individual Assignment\Part 2\data\Yuan Ren Loke.json", "r") as f:
    tmp = json.load(f)

In [106]:
tmp

{'search_metadata': {'id': '6350fbb85fc493037ade6f19',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/3639bf4a998a078d/6350fbb85fc493037ade6f19.json',
  'created_at': '2022-10-20 07:41:44 UTC',
  'processed_at': '2022-10-20 07:41:44 UTC',
  'google_scholar_author_url': 'https://scholar.google.com/citations?user=e1u44asAAAAJ&hl=en&cstart=0&pagesize=1000&sortby=Year',
  'raw_html_file': 'https://serpapi.com/searches/3639bf4a998a078d/6350fbb85fc493037ade6f19.html',
  'total_time_taken': 2.22},
 'search_parameters': {'engine': 'google_scholar_author',
  'author_id': 'e1u44asAAAAJ',
  'hl': 'en',
  'start': 0,
  'num': '1000',
  'sort': 'Year'},
 'author': {'name': 'Yuan Ren Loke',
  'affiliations': 'Nanyang Technological University',
  'email': 'Verified email at ntu.edu.sg',
  'website': 'https://personal.ntu.edu.sg/yrloke/',
  'interests': [{'title': 'Machine Learning',
    'link': 'https://scholar.google.com/citations?view_op=search_authors&hl=en&mauthors=label: